Getting Started with Patra Toolkit

This Colab Notebook is a quickstart guide that helps you:
- Load and preprocess an example image (from an online URL)
- Perform image classification with a pretrained ResNet50 model from PyTorch
- Generate a comprehensive Model Card using the Patra Toolkit

By the end of this tutorial, you will have a Model Card (in JSON format) that captures key metadata about your model and its predictions.

In [ ]:
!pip install torch torchvision patra_toolkit Pillow scikit-learn

### 1.2 Import Dependencies

Below, we import the necessary libraries. We also add the repository root to `sys.path` so that the latest local version of `patra_toolkit` is imported (if needed).

---

In [ ]:
import io
import logging
import requests

import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from PIL import Image

# Import Patra Toolkit components
from patra_toolkit import ModelCard, AIModel

# Set logging level
logging.basicConfig(level=logging.INFO)

---
## 2. Load and Preprocess an Example Image

We'll download an example image from a URL. Then, we'll apply the same preprocessing as required by ResNet50:
- Resize to 256 pixels on the smaller side
- Center-crop to 224×224
- Convert to a tensor and normalize using the ImageNet statistics

---


In [4]:
# URL of an example image (you can choose any valid image URL)
image_path = "/Users/neeleshkarthikeyan/d2i/patra-toolkit/examples/notebooks/data/camera_trap_img.JPG"
image = Image.open(image_path).convert("RGB")

# Define the transformations
from torchvision import transforms

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    # Normalization parameters from ImageNet
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

input_tensor = preprocess(image)
# Create a mini-batch as expected by the model
input_batch = input_tensor.unsqueeze(0)

---
## 3. Image Prediction with Pretrained ResNet50

We load the pretrained ResNet50 model and perform inference on the preprocessed image. We'll then decode the top prediction using the default weights.

---


In [5]:
import os

# Load pretrained ResNet50 model
weights = torchvision.models.ResNet50_Weights.DEFAULT
model = torchvision.models.resnet50(weights=weights)
model.eval()  # set to evaluation mode

# Perform inference
with torch.no_grad():
    output = model(input_batch)

# Get probabilities using softmax
probabilities = F.softmax(output[0], dim=0)

# Get the top 5 predictions
top5_prob, top5_catid = torch.topk(probabilities, 5)

# Use the weights metadata to map category IDs to labels
categories = weights.meta["categories"]

## 4. Model Card Generation with Patra Toolkit

Next, we'll create a Model Card capturing key metadata about the model.
Since we're using a pretrained ResNet50, we'll record its details and top prediction.


In [6]:
# Create a ModelCard instance
mc = ModelCard(
    name="ResNet50",
    version="0.2",
    short_description="A pretrained ResNet50 image classifier",
    full_description="This model card demonstrates using a pretrained ResNet50 model from PyTorch",
    keywords="resnet50, pytorch, image classification, patra, pretrained",
    author="neelk",
    input_type="Image",
    category="classification",
    foundational_model="None",
    citation="https://doi.org/10.48550/arXiv.1512.03385"
)

# Set the input data (image URL)
mc.input_data = ""


### 4.1 Create an AIModel Instance

We attach an AIModel instance to the ModelCard with details about the model.

---

In [7]:
ai_model = AIModel(
    name="ResNet50",
    version="0.1",
    description="Pretrained ResNet50 model from torchvision for image classification.",
    owner="Neelesh Karthikeyan",
    location="",  # will be updated after model submission
    license="BSD-3 Clause",
    framework="pytorch",
    model_type="cnn",
    test_accuracy=0.75
)

# Attach the AIModel to the ModelCard
mc.ai_model = ai_model
mc.populate_requirements()

---
## 6. Submit the Model and Artifact to the Model Store

In [8]:
mc.submit(patra_server_url="http://127.0.0.1:5002",
          model=model,
          file_format="h5",
          model_store="huggingface",
          inference_labels="data/labels.txt",
          artifacts=["data/adult/adult.data",
                     "data/adult/adult.names",
                     "data/adult/adult.test"]
          )

INFO:root:Model card validated successfully.
INFO:root:Model ID retrieved: neelk-resnet50-0.2
INFO:root:Saved PyTorch model as /var/folders/d7/zwq9fkgs65xdfbrv7v00g8dc0000gn/T/neelk-resnet50-0.2.h5
INFO:root:Model serialized successfully.
neelk-resnet50-0.2.h5: 100%|██████████| 103M/103M [00:02<00:00, 38.3MB/s] 
INFO:root:Model uploaded at: https://huggingface.co/patra-iu/neelk-resnet50-0.2/blob/main/neelk-resnet50-0.2.h5
INFO:root:Inference label uploaded at: https://huggingface.co/patra-iu/neelk-resnet50-0.2/blob/main/labels.txt
INFO:root:Artifact 'data/adult/adult.data' uploaded at: https://huggingface.co/patra-iu/neelk-resnet50-0.2/blob/main/adult.data
INFO:root:Artifact 'data/adult/adult.names' uploaded at: https://huggingface.co/patra-iu/neelk-resnet50-0.2/blob/main/adult.names
INFO:root:Artifact 'data/adult/adult.test' uploaded at: https://huggingface.co/patra-iu/neelk-resnet50-0.2/blob/main/adult.test
INFO:root:Model Card submitted successfully.


'success'